In [ ]:
# Import all required packages.
import datetime
import json
import sagemaker

from ipyleaflet import Map, TileLayer, DrawControl, GeoJSON
from sagemaker import Model
import requests

In [1]:
date = datetime.datetime.now().strftime('%Y-%m-%d')

date

'2024-06-05'

In [78]:
# Set constants

BUCKET_NAME = <bucket_name> # Please update this
OPENCAGE_API_KEY = <opencage api key> # please update this

ECR_IMAGE = '637423382292.dkr.ecr.us-west-2.amazonaws.com/llm_test:latest'
NLP_URL = 'https://pnpegu7dupvjvt6alt2yo5nmyi0hgtmj.lambda-url.us-west-2.on.aws/'
ZOOM_LEVEL = 8

# update date with the output of date
PROMPT = """Today's date is <date>. you are given a string with information of an event with time and location. if you can't determine either the time or the location from the query, prepare the error message. you should extract the information from the given text in json format as shown:
{{"datetime": "", "location": "", "error": ""}}

here's the text: {{text}}
json:
"""

In [ ]:
# set runtime client for code execution, and define permissions
sm = sagemaker.Session().sagemaker_runtime_client
sagemaker_role = sagemaker.get_execution_role()

In [ ]:
# Define name of the model
model_name = <identifier> # Please update this.

# Create model based on the ECR_IMAGE and sagemaker role for proper permissions to download files and configuration files.

model = sagemaker.model.Model(
        name=model_name, 
        image_uri=ECR_IMAGE,
        role=sagemaker_role,
        env={
            'PROMPT': PROMPT, 
            'OPENCAGE_API_KEY': OPENCAGE_API_KEY, 
            'BUCKET_NAME': BUCKET_NAME, 
            'LORA_FILENAME': f'{model_name}-adapter_model.safetensors',
            'LORA_CONFIG': f'{model_name}-adapter_config.json'
        }
    )
model.create()

endpoint = model.deploy(
    initial_instance_count=1, 
    instance_type='ml.p3.2xlarge', 
    endpoint_name=model_name, 
    volume_size=50, 
    container_startup_health_check_timeout=600
)

Using already existing model: debug-v2


----

In [136]:
query = "Show me fire scars in maui from august 13, 2023"
response = sm.invoke_endpoint(
    EndpointName=model_name,
    Body=query,
    ContentType="application/json"
)

json.loads(response['Body'].read())

b'{"models":"\\n{\\n  \\"datetime\\": \\"2024-05-30T00:00:00+00:00\\",\\n  \\"location\\": \\"London, UK\\"\\n}"}'

In [ ]:
import requests
import json

# prepare payload for quering using llm and getting responses

query = <update this> # "show me burn scars at maui from august 13, 2023"

payload = json.dumps({
  "query": query,
  "config_path": <your config file path from s3>,
  "model_path": <your model file path from s3>,
  "model_type": "burn_scars",
  "inference_url": <your inference url>,
  "endpoint_name": model_name
})

headers = {
  'Content-Type': 'application/json'
}

# Use deployed app to get inference on the selected date/location
response = requests.request(
      "POST", 
      NLP_URL, 
      headers=headers, 
      data=payload
  )

predictions = response.json()

predictions

In [ ]:
# Setup tile layers to be displayed

datestring = f"{predictions['datetime']}T00:00:00"

HLSL30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_L30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'
HLSS30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_S30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'

In [ ]:
# Identify the central latitude and longitude of the detections
center_lat = sum(predictions['bounding_box'][1::2]) / 2
center_lon = sum(predictions['bounding_box'][0::2]) / 2

# Setup layers for map
hlsl30_tile_layer = TileLayer(url=HLSL30_TILE_LAYER, name='HLSL30', attribution='NASA')
hlss30_tile_layer = TileLayer(url=HLSS30_TILE_LAYER, name='HLSL30', attribution='NASA')

# Instantiate map
detection_map = Map(
        center=(center_lat, center_lon), 
        ZOOM_LEVEL, 
    )
detection_map.add(hlsl30_tile_layer)
detection_map.add(hlss30_tile_layer)
detection_map.add(GeoJSON(data=predictions['geojson']))

detection_map

In [137]:
# Delete all resources after it is used to reduce cost.

model.delete_model()

sess = sagemaker.Session()
sess.delete_endpoint_config(model_name)
sess.delete_endpoint(model_name)